# How to implement custom models

Building a new model in PyTorch Forecasting is relatively easy. Many things are taken care of automatically

* Training, validation and inference is automatically handled for most models - defining the architecture and hyperparameters is sufficient
* Dataloading, normalization, re-scaling etc. is provided by the TimeSeriesDataSet
* Logging training progress with multiple metrics including plotting examples is automatically taken care of
* Masking of entries if different time series have different lengths is automatic

However, there a couple of things to keep in mind if you want to make full use of the package. This tutorial first demonstrates how to implement a simple model and then turns to more complicated implementation scenarios.

We will answer questions such as

* How to transfer an existing PyTorch implementation into PyTorch Forecasting
* How to handle data loading
* How to define and use a custom metric
* How to handle recurrent networks
* How to enable different length time series
* How to deal with covariates

## Building a simple, first model

For demonstration purposes we will choose a simple fully connected model. It takes a timeseries of size `input_size` as input and outputs a new timeseries of size `output_size`. You can think of this `input_size` encoding steps and `output_size` decoding/prediction steps.

In [ ]:
import os

os.chdir("../../..")

In [2]:
import torch
from torch import nn


class FullyConnectedModule(nn.Module):
    def __init__(self, input_size: int, output_size: int, hidden_size: int, n_hidden_layers: int):
        super().__init__()

        # input layer
        module_list = [nn.Linear(input_size, hidden_size), nn.ReLU()]
        # hidden layers
        for _ in range(n_hidden_layers):
            module_list.extend([nn.Linear(hidden_size, hidden_size), nn.ReLU()])
        # output layer
        module_list.append(nn.Linear(hidden_size, output_size))

        self.sequential = nn.Sequential(*module_list)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x of shape: batch_size x n_timesteps_in
        # output of shape batch_size x n_timesteps_out
        return self.sequential(x)


# test that network works as intended
network = FullyConnectedModule(input_size=5, output_size=2, hidden_size=10, n_hidden_layers=2)
x = torch.rand(20, 5)
network(x).shape

torch.Size([20, 2])

In [3]:
from typing import Dict

from pytorch_forecasting.models import BaseModel


class FullyConnectedModel(BaseModel):
    def __init__(self, input_size: int, output_size: int, hidden_size: int, n_hidden_layers: int, **kwargs):
        # saves arguments in signature to `.hparams` attribute, mandatory call - do not skip this
        self.save_hyperparameters()
        # pass additional arguments to BaseModel.__init__, mandatory call - do not skip this
        super().__init__(**kwargs)
        self.network = FullyConnectedModule(
            input_size=self.hparams.input_size,
            output_size=self.hparams.output_size,
            hidden_size=self.hparams.hidden_size,
            n_hidden_layers=self.hparams.n_hidden_layers,
        )

    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # x is a batch generated based on the TimeSeriesDataset
        network_input = x["encoder_cont"].squeeze(-1)
        prediction = self.network(network_input)

        # We need to return a dictionary that at least contains the prediction and the target_scale.
        # The parameter can be directly forwarded from the input.
        return dict(prediction=prediction, target_scale=x["target_scale"])


model = FullyConnectedModel(input_size=5, output_size=2, hidden_size=10, n_hidden_layers=2)

This is a very basic implementation that could be readily used for training. But before we add additional features, let's first have a look how we pass data to this model.

### Passing data to a model

In [4]:
import numpy as np
import pandas as pd

test_data = pd.DataFrame(
    dict(
        value=np.random.rand(30),
        group=np.repeat(np.arange(3), 10),
        time_idx=np.tile(np.arange(10), 3),
    )
)
test_data

,value,group,time_idx
0,0.243177,0,0
1,0.269331,0,1
2,0.596633,0,2
3,0.262185,0,3
4,0.487175,0,4
5,0.441002,0,5
6,0.703872,0,6
7,0.884771,0,7
8,0.671128,0,8
9,0.955309,0,9


In [5]:
from pytorch_forecasting import TimeSeriesDataSet

# create the dataset from the pandas dataframe
dataset = TimeSeriesDataSet(
    test_data,
    group_ids=["group"],
    target="value",
    time_idx="time_idx",
    min_encoder_length=5,
    max_encoder_length=5,
    min_prediction_length=2,
    max_prediction_length=2,
    time_varying_unknown_reals=["value"],
)

In [6]:
dataset.get_parameters()

{'time_idx': 'time_idx',
 'target': 'value',
 'group_ids': ['group'],
 'weight': None,
 'max_encoder_length': 5,
 'min_encoder_length': 5,
 'min_prediction_idx': 0,
 'min_prediction_length': 2,
 'max_prediction_length': 2,
 'static_categoricals': [],
 'static_reals': [],
 'time_varying_known_categoricals': [],
 'time_varying_known_reals': [],
 'time_varying_unknown_categoricals': [],
 'time_varying_unknown_reals': ['value'],
 'variable_groups': {},
 'dropout_categoricals': [],
 'constant_fill_strategy': {},
 'allow_missings': False,
 'add_relative_time_idx': False,
 'add_target_scales': False,
 'add_encoder_length': False,
 'target_normalizer': GroupNormalizer(transformation='relu'),
 'categorical_encoders': {'__group_id__group': NaNLabelEncoder(),
  'group': NaNLabelEncoder()},
 'scalers': {'value': GroupNormalizer(transformation='relu')},
 'randomize_length': None,
 'predict_mode': False}

Now, we take a look at the output of the dataloader. It's `x` will be fed to the model's forward method, that is why it is so important to understand it.

In [7]:
# convert the dataset to a dataloader
dataloader = dataset.to_dataloader(batch_size=4)

# and load the first batch
x, y = next(iter(dataloader))
print("x =", x)
print("\ny =", y)
print("\nsizes of x =")
for key, value in x.items():
    print(f"\t{key} = {value.size()}")

x = {'encoder_cat': tensor([], size=(4, 5, 0), dtype=torch.int64), 'encoder_cont': tensor([[[-0.5022],
         [-0.3726],
         [ 0.2286],
         [-0.9914],
         [-1.4670]],

        [[-0.6323],
         [-0.5361],
         [ 0.6683],
         [-0.5624],
         [ 0.2655]],

        [[-0.2086],
         [ 0.4872],
         [-1.4002],
         [-0.4996],
         [ 0.8897]],

        [[ 0.4872],
         [-1.4002],
         [-0.4996],
         [ 0.8897],
         [-0.9519]]]), 'encoder_target': tensor([[0.2785, 0.3138, 0.4771, 0.1456, 0.0163],
        [0.2432, 0.2693, 0.5966, 0.2622, 0.4872],
        [0.3583, 0.5474, 0.0345, 0.2793, 0.6568],
        [0.5474, 0.0345, 0.2793, 0.6568, 0.1563]]), 'encoder_lengths': tensor([5, 5, 5, 5]), 'decoder_cat': tensor([], size=(4, 2, 0), dtype=torch.int64), 'decoder_cont': tensor([[[ 0.1337],
         [ 1.7081]],

        [[ 0.0956],
         [ 1.0629]],

        [[-0.9519],
         [-1.0224]],

        [[-1.0224],
         [ 2.1368]]]), 

This explains why we had to first extract the correct input in our simple `FullyConnectedModel` above before passing it to our `FullyConnectedModule`.
As a reminder:
       

In [8]:
def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    # x is a batch generated based on the TimeSeriesDataset
    network_input = x["encoder_cont"].squeeze(-1)
    prediction = self.network(network_input)

    # We need to return a dictionary that at least contains the prediction and the target_scale.
    # The parameter can be directly forwarded from the input.
    return dict(prediction=prediction, target_scale=x["target_scale"])

For such a simple architecture, we can ignore most of the inputs in ``x``. You do not have to worry about moving tensors to specifc GPUs, [PyTorch Lightning](https://pytorch-lightning.readthedocs.io) will take care of this for you.

Now, let's check if our model works:

In [9]:
x, y = next(iter(dataloader))
model(x)

{'prediction': tensor([[ 0.2248, -0.0798],
         [ 0.2240, -0.0938],
         [ 0.2256, -0.0776],
         [ 0.2165, -0.0346]], grad_fn=<AddmmBackward>),
 'target_scale': tensor([[0.4150, 0.2718],
         [0.4150, 0.2718],
         [0.4150, 0.2718],
         [0.4150, 0.2718]])}

In [10]:
dataset.x_to_index(x)

,time_idx,group
0,6,0
1,8,0
2,8,2
3,6,2


### Coupling datasets and models

In [11]:
class FullyConnectedModel(BaseModel):
    def __init__(self, input_size: int, output_size: int, hidden_size: int, n_hidden_layers: int, **kwargs):
        # saves arguments in signature to `.hparams` attribute, mandatory call - do not skip this
        self.save_hyperparameters()
        # pass additional arguments to BaseModel.__init__, mandatory call - do not skip this
        super().__init__(**kwargs)
        self.network = FullyConnectedModule(
            input_size=self.hparams.input_size,
            output_size=self.hparams.output_size,
            hidden_size=self.hparams.hidden_size,
            n_hidden_layers=self.hparams.n_hidden_layers,
        )

    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # x is a batch generated based on the TimeSeriesDataset
        network_input = x["encoder_cont"].squeeze(-1)
        prediction = self.network(network_input).unsqueeze(-1)

        # We need to return a dictionary that at least contains the prediction and the target_scale.
        # The parameter can be directly forwarded from the input.
        return dict(prediction=prediction, target_scale=x["target_scale"])

    @classmethod
    def from_dataset(cls, dataset: TimeSeriesDataSet, **kwargs):
        new_kwargs = {
            "output_size": dataset.max_prediction_length,
            "input_size": dataset.max_encoder_length,
        }
        new_kwargs.update(kwargs)  # use to pass real hyperparameters and override defaults set by dataset
        # example for dataset validation
        assert dataset.max_prediction_length == dataset.min_prediction_length, "Decoder only supports a fixed length"
        assert dataset.min_encoder_length == dataset.max_encoder_length, "Encoder only supports a fixed length"
        assert (
            len(dataset.time_varying_known_categoricals) == 0
            and len(dataset.time_varying_known_reals) == 0
            and len(dataset.time_varying_unknown_categoricals) == 0
            and len(dataset.static_categoricals) == 0
            and len(dataset.static_reals) == 0
            and len(dataset.time_varying_unknown_reals) == 1
            and dataset.time_varying_unknown_reals[0] == dataset.target
        ), "Only covariate should be the target in 'time_varying_unknown_reals'"

        return super().from_dataset(dataset, **new_kwargs)

Now, let's initialize from our dataset:

In [12]:
model = FullyConnectedModel.from_dataset(dataset, hidden_size=10, n_hidden_layers=2)
model.summarize("full")  # print model summary
model.hparams


   | Name                 | Type                 | Params
---------------------------------------------------------------
0  | loss                 | SMAPE                | 0     
1  | logging_metrics      | ModuleList           | 0     
2  | network              | FullyConnectedModule | 302   
3  | network.sequential   | Sequential           | 302   
4  | network.sequential.0 | Linear               | 60    
5  | network.sequential.1 | ReLU                 | 0     
6  | network.sequential.2 | Linear               | 110   
7  | network.sequential.3 | ReLU                 | 0     
8  | network.sequential.4 | Linear               | 110   
9  | network.sequential.5 | ReLU                 | 0     
10 | network.sequential.6 | Linear               | 22    


"hidden_size":                10
"input_size":                 5
"learning_rate":              0.001
"log_gradient_flow":          False
"log_interval":               -1
"log_val_interval":           -1
"logging_metrics":            ModuleList()
"loss":                       SMAPE()
"monotone_constaints":        {}
"n_hidden_layers":            2
"optimizer":                  ranger
"output_size":                2
"output_transformer":         GroupNormalizer(transformation='relu')
"reduce_on_plateau_min_lr":   1e-05
"reduce_on_plateau_patience": 1000
"weight_decay":               0.0

### Defining additional hyperparameters

In [13]:
model.hparams

"hidden_size":                10
"input_size":                 5
"learning_rate":              0.001
"log_gradient_flow":          False
"log_interval":               -1
"log_val_interval":           -1
"logging_metrics":            ModuleList()
"loss":                       SMAPE()
"monotone_constaints":        {}
"n_hidden_layers":            2
"optimizer":                  ranger
"output_size":                2
"output_transformer":         GroupNormalizer(transformation='relu')
"reduce_on_plateau_min_lr":   1e-05
"reduce_on_plateau_patience": 1000
"weight_decay":               0.0

In [14]:
print(BaseModel.__init__.__doc__)


        BaseModel for timeseries forecasting from which to inherit from

        Args:
            log_interval (Union[int, float], optional): Batches after which predictions are logged. If < 1.0, will log
                multiple entries per batch. Defaults to -1.
            log_val_interval (Union[int, float], optional): batches after which predictions for validation are
                logged. Defaults to None/log_interval.
            learning_rate (float, optional): Learning rate. Defaults to 1e-3.
            log_gradient_flow (bool): If to log gradient flow, this takes time and should be only done to diagnose
                training failures. Defaults to False.
            loss (Metric, optional): metric to optimize. Defaults to SMAPE().
            logging_metrics (nn.ModuleList[MultiHorizonMetric]): list of metrics that are logged during training.
                Defaults to [].
            reduce_on_plateau_patience (int): patience after which learning rate is reduced by a

## Using covariates

In [15]:
from pytorch_forecasting.models.base_model import BaseModelWithCovariates

print(BaseModelWithCovariates.__doc__)


    Model with additional methods using covariates.

    Assumes the following hyperparameters:

    Args:
        static_categoricals (List[str]): names of static categorical variables
        static_reals (List[str]): names of static continuous variables
        time_varying_categoricals_encoder (List[str]): names of categorical variables for encoder
        time_varying_categoricals_decoder (List[str]): names of categorical variables for decoder
        time_varying_reals_encoder (List[str]): names of continuous variables for encoder
        time_varying_reals_decoder (List[str]): names of continuous variables for decoder
        x_reals (List[str]): order of continuous variables in tensor passed to forward function
        x_categoricals (List[str]): order of categorical variables in tensor passed to forward function
        embedding_sizes (Dict[str, Tuple[int, int]]): dictionary mapping categorical variables to tuple of integers
            where the first integer denotes the nu

In [16]:
from typing import Dict, List, Tuple

from pytorch_forecasting.models.nn import MultiEmbedding


class FullyConnectedModelWithCovariates(BaseModelWithCovariates):
    def __init__(
        self,
        input_size: int,
        output_size: int,
        hidden_size: int,
        n_hidden_layers: int,
        x_reals: List[str],
        x_categoricals: List[str],
        embedding_sizes: Dict[str, Tuple[int, int]],
        embedding_labels: Dict[str, List[str]],
        static_categoricals: List[str],
        static_reals: List[str],
        time_varying_categoricals_encoder: List[str],
        time_varying_categoricals_decoder: List[str],
        time_varying_reals_encoder: List[str],
        time_varying_reals_decoder: List[str],
        embedding_paddings: List[str],
        categorical_groups: Dict[str, List[str]],
        **kwargs,
    ):
        # saves arguments in signature to `.hparams` attribute, mandatory call - do not skip this
        self.save_hyperparameters()
        # pass additional arguments to BaseModel.__init__, mandatory call - do not skip this
        super().__init__(**kwargs)

        # create embedder - can be fed with x["encoder_cat"] or x["decoder_cat"] and will return
        # dictionary of category names mapped to embeddings
        self.input_embeddings = MultiEmbedding(
            embedding_sizes=self.hparams.embedding_sizes,
            categorical_groups=self.hparams.categorical_groups,
            embedding_paddings=self.hparams.embedding_paddings,
            x_categoricals=self.hparams.x_categoricals,
            max_embedding_size=self.hparams.hidden_size,
        )

        # calculate the size of all concatenated embeddings + continous variables
        n_features = sum(
            embedding_size for classes_size, embedding_size in self.hparams.embedding_sizes.values()
        ) + len(self.reals)

        # create network that will be fed with continious variables and embeddings
        self.network = FullyConnectedModule(
            input_size=self.hparams.input_size * n_features,
            output_size=self.hparams.output_size,
            hidden_size=self.hparams.hidden_size,
            n_hidden_layers=self.hparams.n_hidden_layers,
        )

    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # x is a batch generated based on the TimeSeriesDataset
        batch_size = x["encoder_lengths"].size(0)
        embeddings = self.input_embeddings(x["encoder_cat"])  # returns dictionary with embedding tensors
        network_input = torch.cat(
            [x["encoder_cont"]]
            + [
                emb
                for name, emb in embeddings.items()
                if name in self.encoder_variables or name in self.static_variables
            ],
            dim=-1,
        )
        prediction = self.network(network_input.view(batch_size, -1))

        # We need to return a dictionary that at least contains the prediction and the target_scale.
        # The parameter can be directly forwarded from the input.
        return dict(prediction=prediction, target_scale=x["target_scale"])

    @classmethod
    def from_dataset(cls, dataset: TimeSeriesDataSet, **kwargs):
        new_kwargs = {
            "output_size": dataset.max_prediction_length,
            "input_size": dataset.max_encoder_length,
        }
        new_kwargs.update(kwargs)  # use to pass real hyperparameters and override defaults set by dataset
        # example for dataset validation
        assert dataset.max_prediction_length == dataset.min_prediction_length, "Decoder only supports a fixed length"
        assert dataset.min_encoder_length == dataset.max_encoder_length, "Encoder only supports a fixed length"

        return super().from_dataset(dataset, **new_kwargs)

Let us create a new dataset with categorical variables and see how the model can be instantiated from it.

In [17]:
import numpy as np
import pandas as pd

from pytorch_forecasting import TimeSeriesDataSet

test_data_with_covariates = pd.DataFrame(
    dict(
        # as before
        value=np.random.rand(30),
        group=np.repeat(np.arange(3), 10),
        time_idx=np.tile(np.arange(10), 3),
        # now adding covariates
        categorical_covariate=np.random.choice(["a", "b"], size=30),
        real_covariate=np.random.rand(30),
    )
).astype(
    dict(group=str)
)  # categorical covariates have to be of string type
test_data_with_covariates

,value,group,time_idx,categorical_covariate,real_covariate
0,0.736227,0,0,a,0.038764
1,0.504614,0,1,b,0.114938
2,0.313007,0,2,b,0.738702
3,0.967045,0,3,a,0.705307
4,0.108684,0,4,a,0.519999
5,0.794994,0,5,b,0.756494
6,0.066759,0,6,b,0.968352
7,0.859615,0,7,a,0.284139
8,0.885078,0,8,a,0.540724
9,0.285903,0,9,a,0.363773


In [18]:
# create the dataset from the pandas dataframe
dataset_with_covariates = TimeSeriesDataSet(
    test_data_with_covariates,
    group_ids=["group"],
    target="value",
    time_idx="time_idx",
    min_encoder_length=5,
    max_encoder_length=5,
    min_prediction_length=2,
    max_prediction_length=2,
    time_varying_unknown_reals=["value"],
    time_varying_known_reals=["real_covariate"],
    time_varying_known_categoricals=["categorical_covariate"],
    static_categoricals=["group"],
)

model = FullyConnectedModelWithCovariates.from_dataset(dataset_with_covariates, hidden_size=10, n_hidden_layers=2)
model.summarize("full")  # print model summary
model.hparams


   | Name                                              | Type                 | Params
--------------------------------------------------------------------------------------------
0  | loss                                              | SMAPE                | 0     
1  | logging_metrics                                   | ModuleList           | 0     
2  | input_embeddings                                  | MultiEmbedding       | 11    
3  | input_embeddings.embeddings                       | ModuleDict           | 11    
4  | input_embeddings.embeddings.group                 | Embedding            | 9     
5  | input_embeddings.embeddings.categorical_covariate | Embedding            | 2     
6  | network                                           | FullyConnectedModule | 552   
7  | network.sequential                                | Sequential           | 552   
8  | network.sequential.0                              | Linear               | 310   
9  | network.sequential.1           

"categorical_groups":                {}
"embedding_labels":                  {'group': {'0': 0, '1': 1, '2': 2}, 'categorical_covariate': {'a': 0, 'b': 1}}
"embedding_paddings":                []
"embedding_sizes":                   {'group': [3, 3], 'categorical_covariate': [2, 1]}
"hidden_size":                       10
"input_size":                        5
"learning_rate":                     0.001
"log_gradient_flow":                 False
"log_interval":                      -1
"log_val_interval":                  -1
"logging_metrics":                   ModuleList()
"loss":                              SMAPE()
"monotone_constaints":               {}
"n_hidden_layers":                   2
"optimizer":                         ranger
"output_size":                       2
"output_transformer":                GroupNormalizer(transformation='relu')
"reduce_on_plateau_min_lr":          1e-05
"reduce_on_plateau_patience":        1000
"static_categoricals":               ['group']
"stati

To test that the model could be trained, pass a sample batch.

In [19]:
x, y = next(iter(dataset_with_covariates.to_dataloader(batch_size=4)))  # generate batch
model(x)  # pass batch through model

{'prediction': tensor([[-0.0016, -0.1305],
         [-0.0012, -0.1344],
         [ 0.0007, -0.1265],
         [-0.0048, -0.1355]], grad_fn=<AddmmBackward>),
 'target_scale': tensor([[0.5142, 0.2994],
         [0.5142, 0.2994],
         [0.5142, 0.2994],
         [0.5142, 0.2994]])}

## Implementing a recurrent model

## Defining and using a custom metric

## Adding custom plotting and interpretation